In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
movies= pd.read_csv("../datasets/movies Recommendations dataset/movies/movies.csv",usecols=["movieId","title"],index_col=["movieId"])
rating= pd.read_csv("../datasets/movies Recommendations dataset/movies/ratings.csv",usecols=["userId","movieId","rating"])

In [3]:
rating.head(2)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0


In [4]:
movies_users =rating.pivot(index="movieId",columns="userId",values="rating").fillna(0)

In [5]:
movies_users.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
from scipy.sparse import csr_matrix

In [7]:
mat_movies=csr_matrix(movies_users.values)

In [28]:
mat_movies[2]

<1x610 sparse matrix of type '<class 'numpy.float64'>'
	with 52 stored elements in Compressed Sparse Row format>

In [8]:
from sklearn.neighbors import NearestNeighbors

In [9]:
#NearestNeighbors?

In [10]:
model=NearestNeighbors(metric='cosine',algorithm='brute',n_neighbors=20)
model.fit(movies_users)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=20, p=2,
                 radius=1.0)

In [14]:
from fuzzywuzzy import process

In [54]:
distance,indecies=model.kneighbors(mat_movies[2],n_neighbors=3)
for i in indecies:
    print(movies["title"].reindex(i))

movieId
2       Jumanji (1995)
2578               NaN
607                NaN
Name: title, dtype: object


In [61]:
def recommender(movie_name,data,n):
    idx= process.extractOne(movie_name,movies["title"])[2]
    print("movie selescted is :", movies["title"][idx] ,"indx : " ,idx)
    distance,indecies=model.kneighbors(data[idx],n_neighbors=n)
    print("recommended movies")
    for i in indecies:
        print(movies["title"].reindex(i).where(i!=idx))

In [62]:
recommender("the matrix",mat_movies,10)

movie selescted is : Father of the Bride Part II (1995) indx :  5
recommended movies
movieId
5                                   NaN
592                       Batman (1989)
31               Dangerous Minds (1995)
254           Jefferson in Paris (1995)
15              Cutthroat Island (1995)
520    Robin Hood: Men in Tights (1993)
398                                 NaN
43                   Restoration (1995)
84                                  NaN
136                                 NaN
Name: title, dtype: object
